# Project

There are three tasks in this project. Each task presents a specific question for you to answer. Submit a Jupyter Notebook file that contains the code and results of your analysis. Describe your overall approach clearly in plain text. Comment your code carefully to demonstrate that you understand how to implement the analysis.

Write your solutions to the given cells (see "WRITE YOUR CODE HERE"). Make your code print only what is requested. Select `Kernel` &rightarrow; `Restart & Clear Output` and then `Cell` &rightarrow; `Run All` from the menu before submitting your solutions, so that your submission contains clean output produced from scratch.

Each task is worth 20 points. You must submit a correct and clearly documented analysis to get full points. Partial points can be obtained by successfully performing and/or documenting essential parts of the analysis. Documentation (i.e. descriptions and comments) is particularly important if your implementation is not fully correct: you can get partial points for explaining your objectives without implementing them correctly or at all.


## Task 1

What domain architectures do the seven known human G protein-coupled receptor kinases have? How many of each architecture are there? Print the architectures together with their counts, one per line.

Suggested steps:
- Get the proteins encoded by the GRK1 - GRK7 genes from UniProt.
- Extract the domains from the entries.
- Find and count the domain architectures.

A domain architecture is an ordered arrangement of domains. An architecture can be printed nicely by joining its domain names with a separator, e.g. a dash.


### Task 1: report


To found out the Uniprot ID's for the genes, a manual search to Uniprot.org database was done. A search with each gene name (GRK1 to GRK7) was done and with each gene the corresponding Uniprot ID was picked. 

#### 2.3. -21: Please see the update below the solution.

These ID's were then used as a part of the search to the Uniprot mapping service. This, and all the following searches, were done with the Pythons requests module that allow user to send HTTP/1.1 requests (https://pypi.org/project/requests/).

To work in a logistically efficient way, it is reasonable to get all the given genes with a on request. To achieve this, one can use the Uniprot mapping to map within the Uniprot database and get data for the given genes. However the mapping only works with Uniprot ID's and/or names

The data accessed via requests was written to an XML format to be accessed locally. The file was parsed to be used in Python by using an additional module called Biopython (https://biopython.org/). Briefly described; the Biopython contains several submodules that user can use to analyze biological data. All of the analysis of this project were done using various Biopython modules.

The XML file parsed with Biopythons Bio.SeqIO contains all the information of the GRK genes. The information is stored in special fields that have describing headers (features). However the downside is that when iterating the fields user might get names dublicated.  

To prevent name (key) dublications and to store data in efficient way, the dictionary module of the Python was used. The name of the GRKs (as given in Uniprot) were used as keys for the dictionary and the domains of the given protein as values for the dictionary. This made possible to get a compact view of the domain architectures of the GRKs and to count the shared domains.

The seven GRK's have the following domains (ordered in name + count): RGS 7, Protein kinase 7, AGC-kinase C-terminal 7 and PH 2.


In [2]:
# To get all the applications working, Python must download the module(s) of the application(s) first
# Modules were renamed for more convenient use.
import Bio.Align as BA
import Bio.Align.Applications as BAA
import Bio.AlignIO as BAIO
import Bio.Blast.NCBIWWW as BBNW
import Bio.Seq as BS
import Bio.SeqRecord as BSR
import Bio.SeqIO as BSIO
import requests as R

In [4]:
# API address
url = 'https://www.uniprot.org/uploadlists/'

# required parameters as dictionary
data = {
    # map within Uniprot, to AC/ID
    'from': 'ACC+ID',
    # to Uniprot AC
    'to': 'ACC',
    # output format is xml as domain info 
    # is needed from the info fields
    'format': 'xml',
    # Uniprot IDs of the seven genes
    'query': 'Q15835 P25098 P35626 P32298 P34947 P43250 Q8WTQ7',
    # mapping toimii: from: GENENAME to ACC+ID, organism: "homo sapiens"
    # https://www.uniprot.org/help/api_idmapping
}

# send query and get response
response = R.get(url, params=data)

#save the response to a gprotdomains.xml -file
with open('gprotdomains.xml', 'w') as f:
    f.write(response.text)

In [3]:
# The following code is from example solution to ex 3.8 
# in the course BIOI4270 provided by teacher Juho Heimonen

# It is used for simplicity and for correctness, as the author
# couldn't solve the ex 3.8 during the course


# a new empty dictionary to collect the domain architecture
domains = {}

# as the file contains multiple entries use Bio.SeqIO.parse
# to make an iteratable SeqRecord object

for x in BSIO.parse("gprotdomains.xml", "uniprot-xml"):
    #start iterating with iterator x

    for y in x.features:
        #from the feature level another iteration is made

        if y.type == "domain":
        #if the type section in features contain word domain, do following

                # check if the key (name of the protein) is not yet present
                # in the dictionary keys
                if not x.name in domains:

                    #if naem not found (true): initalize the keys as entry names and values as empty lists
                    domains[x.name] = []

                    # inside the empty values list
                    # append the domain descriptions for each protein
                domains[x.name].append(y.qualifiers["description"])


In [5]:
# From the formatted dictionary (domains)
for key,value in domains.items():
    # get the domain architectures represented as key value pairs
    # where keys are the names and the values are the domains of the gene
    print(key,value)

    # Print an empty space to give a better visibility
print()

# as the domains are in nested list, counter can't acces them straightly
# therefore combine the entries into one list

# an empty list for collecting the entries
allDom = []

# start iterating the entries --> acces the lists 
for things in domains.values():

    #start iterating the key-value pairs inside lists
        for value in things:

            #append them to the empty list
            allDom.append(value)

#import automatic counter to count the occurences of the domains
from collections import Counter

# as the Counter produces a dictionary, 
# acces keys (domain names) and values (number of occurences)
for key,value in Counter(allDom).items():
    # and print them
    print(key,value)

GRK1_HUMAN ['RGS', 'Protein kinase', 'AGC-kinase C-terminal']
ARBK1_HUMAN ['RGS', 'Protein kinase', 'AGC-kinase C-terminal', 'PH']
ARBK2_HUMAN ['RGS', 'Protein kinase', 'AGC-kinase C-terminal', 'PH']
GRK4_HUMAN ['RGS', 'Protein kinase', 'AGC-kinase C-terminal']
GRK5_HUMAN ['RGS', 'Protein kinase', 'AGC-kinase C-terminal']
GRK6_HUMAN ['RGS', 'Protein kinase', 'AGC-kinase C-terminal']
GRK7_HUMAN ['RGS', 'Protein kinase', 'AGC-kinase C-terminal']

RGS 7
Protein kinase 7
AGC-kinase C-terminal 7
PH 2


### Task1 UPDATE 2.3.-21:

After the Q&A session in 26.2. I realised I could've done the search by making a query with "gene: GRK1..." etc.

I also noticed a possibility in the ID Mapping section to map with gene names to the Swissprot database. From the online server the query was a piece of cake, but for some reason I couldn't get the query to work via Python. Hence I left everything as it was.

If you could have a look at my examples below and be kind enough to tell me what is wrong with them. Every time I got the query to work, I got all animalias GRK genes whilst having the filter 'organism': "Homo sapiens (Human) [9606]"


In [ ]:
# API address
import requests

url = 'https://www.uniprot.org/uploadlists/'

# required parameters as dictionary
data = {
    # map with Gene names
    'from': 'GENENAME',
    # to Uniprot SWISSPROT
    'to': 'SWISSPROT',
    #'reviewed': 'yes',
    # Uniprot IDs of the seven genes
    'organism': "Homo sapiens (Human) [9606]",
    'query': 'GRK1 GRK2 GRK3 GRK4 GRK5 GRK6 GRK7', 
    
    # ^ that gives me every animal with GRK's
    'format': 'xml'

    #nothing: 'query': 'GRK1 GRK2 GRK3 GRK4 GRK5 GRK6 GRK7 AND organism: Homo sapiens (Human) [9606]'
    #none 'query': 'GRK1 GRK2 GRK3 GRK4 GRK5 GRK6 GRK7 AND organism: "Homo sapiens (Human) [9606]"', 
    #none: 'query': '"GRK1 GRK2 GRK3 GRK4 GRK5 GRK6 GRK7" AND organism: "Homo sapiens (Human) [9606]"'
    }
# send query and get response
response = requests.get(url, params=data)
print(response.url)

## Task 2

The human histone H3.1, H3.2, H3.3, H3.4 and H3.5 proteins have equal lengths and are mostly identical. At which alignment positions does at least one of them differ from the others? Print each such position together with its most common residue, one per line. Treat gaps as different from all residues.

Suggested steps:
- Get the UniProt entries P68431, Q71DI3, P84243, Q16695 and Q6NXT2.
- Create a multiple sequence alignment.
- Scan the alignment for non-100%-identical positions.


### Task2: report

Uniprot database contains the information of given gene in compact FASTA- format. The format is used to describe the nucleotide or amino acid (AA) structure in letter format, where every letter represents a single AA or nucleotide. This format can be used to make different analysis and annotations. 

In this task, the AA sequence was collected from the Uniprot via Biopython. The FASTAs were accessed directly and written into a separate file. 

To obtain the information about the similarity of the sequences, a multiple sequence alignment was done. Described extremely shortly; in the MSA the sequences are put align and a best possible similarity is tried to obtain. The similarity is calculated by different algorithms and scoring methods.

Various MSA algorithms/programs exist, but in this project the MUltiple Sequence Comparison by Log-Expectation (MUSCLE) program was used. The detailled description of MUSCLE is found in https://doi.org/10.1186/1471-2105-5-113.

The Biopython needs an external program to do the alignment and as said, the MUSCLE was chosen for the task. The FASTA -file containing the structures of the histones was aligned with MUSCLE and the MSA file was saved for further use. After using MUSCLE, Bio.AlignIO module was used to parse the file into Bio.Align.MultipleSeqAlignment object. From the parsed object, a Counter object from Pythons collections module was used to get the unidentical positions. The final conclusion was that from 136 positions 14 are unidentical.

In [ ]:

# API address
url = 'https://www.uniprot.org/uploadlists/'

# required parameters as dictionary
data = {
    # map within Uniprot, to AC/ID
    'from': 'ACC+ID',
    # to Uniprot AC
    'to': 'ACC',
    # output format is fasta, as only structure was wanted
    'format': 'fasta',
    # Uniprot IDs of the Histones
    'query': 'P68431 Q71DI3 P84243 Q16695 Q6NXT2',
    #P68431, Q71DI3, P84243, Q16695 and Q6NXT2
}

# send query and get response
response = R.get(url, params=data)

#save the response to a fasta formatted file named histonefasta 
with open('histonefasta.fasta', 'w') as f:
    f.write(response.text)



In [3]:
# the Biopython doesn't have an ability to do MSA's
# therefore it's required to use an external program to do it 

# For the sake of convenience, the Muscle program was use in the exercise
# as it was used in the lectures. 

# Author of this work tried (for fun) to install Clustal Omega, 
# but gave up after some time

# name of the program to use (muscle)
# runs from the same folder, no need to give addres
program_path = 'muscle.exe'
# check that the executable exists
import os.path as OP
assert OP.isfile(program_path), "MUSCLE executable missing"

# setup MUSCLE
# # read unaligned file and write the aligned sequences to a new file 
cmd = BAA.MuscleCommandline(cmd=program_path,
                            input="histonefasta.fasta",
                            out="alignedhistones.fasta")
# run MUSCLE
stdout, stderr = cmd()


In [135]:
# Slightly offtopic, but it is good to eat once in a while
# Helps to notice what one has done in silly ways

# Kiitos neuvoistasi!!!

# read a single alignment from a file in the FASTA format 
alignment = BAIO.read("alignedhistones.fasta", "fasta")
# do not parse a parsed file, like someone tried to do...

# create a range of numbers starting from 0 
# ending to the last place of the MSA
# x represents the value and increases +1 every time
for x in range(0,alignment.get_alignment_length()):
    
    # x gives the column to be splitted  
    if 5 not in Counter(alignment[:,x]).values():

        # since one knows that there are 5 genes to compare
        # one can conclude that if the value (number of hits)
        # is 5, sequences are identical
        # hence one can separate the non identicals (value not 5) to be printed 

        # x is the index in Python, +1 to get the actual position in the sequence
        print("Position number: "+str(x+1))
        # Handy way to get the most common residue: using Counter.most_common(1)
        print("Most common residue: %s" % Counter(alignment[:,x]).most_common(1))
        print()
        # String formatting to get list concatenated with string and get a space between the groups

Position number: 25
Most common residue: [('A', 4)]

Position number: 30
Most common residue: [('A', 4)]

Position number: 32
Most common residue: [('A', 3)]

Position number: 34
Most common residue: [('G', 4)]

Position number: 37
Most common residue: [('K', 4)]

Position number: 72
Most common residue: [('V', 4)]

Position number: 80
Most common residue: [('K', 4)]

Position number: 88
Most common residue: [('S', 3)]

Position number: 90
Most common residue: [('V', 4)]

Position number: 91
Most common residue: [('M', 3)]

Position number: 97
Most common residue: [('S', 3)]

Position number: 99
Most common residue: [('A', 4)]

Position number: 105
Most common residue: [('F', 4)]

Position number: 112
Most common residue: [('A', 4)]



## Task 3

The file `peptide.fasta` contains a hypothetical peptide sequence in the FASTA format. Based on the Gene Ontology terms associated with similar sequences in UniProt, what molecular functions could this protein have? Print the IDs of Gene Ontology terms together with their counts and human-friendly names, one per line and sorted in descending order by count.

Suggested steps:
- Run BLAST and get hits with $E < 0.001$.
- Get the corresponding UniProt entries.
- Collect the Gene Ontology terms from the entries.


### Task3: report

As the idea was to collect the corresponding Uniprot terms, it might be preferrable to do the BLAST against the Swissprot database. The Swissprot contains manually annotated entries in the Uniprot KB database and thus user can easily collect the Uniprot ID's straight from the query. Uniprot can handle the NCBI terms as well, but one ID might produce several hits and the NCBI ID's might have to be translated to correspond the Uniprot.

To figure out previously unknown organism structure, a BLAST search is - if not the best way - but one of the best ways to collect information of the structure. BLAST method can be briefly described as a search based on scores and local alignment. Using pre-defined parameters, the BLAST slices the structure (FASTA seq) to smaller pieces and starts searching for matches from database. Then based on a scoring matrice, the algorithm calculates a score for the given match and tries to lengthen the alignment. After acquiring the "relatives", the BLAST then calculates score and e-value (Expect Value) for the whole alignment. (Wheeler et.al, 2007, https://www.ncbi.nlm.nih.gov/books/NBK1734/)

There are various BLAST databases to use, but the Biopythons Bio.Blast.NCBIWWW module uses the NCBI's BLAST database. It is possible for the user to modify  the parameters of the search, for this project the default parameters were used. BLAST used the “blosum62” matrix to give score for the matches and word size 3 to split the sequences. To be accepted for further analysis the e-value (how likely is it that similarly scored match would come randomly) is used for filtering the hits. The range of generally used E's varies from 0.001 to 0.0000001, in this project E of 0.001 was used. (Wheeler et.al, 2007, https://www.ncbi.nlm.nih.gov/books/NBK1734/)

After making the query to Swissprot, the queryfile was written to a file and saved locally for further use. To interpret the results the file was made Biopython compatible with the Bio.SearchIO module. The results were filtered and accession numbers extracted to form an Uniprot query. After using the filter e < 0.001, 10 peptides remained to be used in the Uniprot query: P0ACS5 Q9HV30 Q93CH6 Q8Z8S3 Q8ZCA8 Q8FK74 Q8XD09 P0A9G4 P0A2Q8 Q9X5V4. The Uniprot query file was again saved and parsed to Biopython compatible with Bio.SeqIO. From the parsed file, isolation of the GO terms yielded 9 unique GO terms, listed below the code.

The Gene Ontology is a large database to describe the genetic functions. The Biopython doesn't support the GO per se, but the GO can be downoladed (http://purl.obolibrary.org/obo/go.obo) to be imported externally via spesific parsing method. The parsing method was given previously on the course and the go.obo file was also downoladed previously. The query to find out the molecular functions for the 10 peptides was rather simple, the GO was accessed one by one with the GO terms isolated from the Uniprot file.

The evidence suggest the unknown sequence to be from a bacteria and that the peptide might have transcriptional properties. It might act as a regulator in the transcription, this is supported by the the zinc binding and protein binding abilities. 

As the sequence seems to be bacteria based there are few hyopthesis one can make from the Cu2+ binding abilities. The cuprous ion is not well tolerated with bacterias and is preferrably extracted from the cytoplasm to periplasm or out of the cell. When the amount of Cu2+ exceeds some spesific treshold repressor proteins bind to copper and detachs from genes encoding chaperones and P-type ATPases. This enhances the Cu2+ elimination and proofens the bacterias livability. (Festa & Thiele, 2011, https://doi.org/10.1016/j.cub.2011.09.040) 

Based on the gathered evidence from bacteria origin to transcription activites/DNA binding abilites and cuprous binding ablities, I'd make a hypothesis this unknown sequence to act as repressor protein for genes involved to cuprous ion elimination in bacteria.


In [ ]:
with open('peptide.fasta') as f:
    fasta = f.read()
    query = fasta

    
# BLAST program to use
program = "blastp"
# database to search against
database = "swissprot"
# query sequence as a Seq object (the read fasta)
query = query


handle = BBNW.qblast(program, database, query)
with open('blastedPeptides.blast', 'w') as f:
    f.write(handle.read())
    

In [10]:
# for some reason Bio Search IO import had to be here
# didn't work fron the start
import Bio.SearchIO as BSeIO

# determining the filter: e < 0.001
fn  = lambda hsp: hsp.evalue < 0.001

# open the file
# parse a query result into a QueryResult object
with open('blastedPeptides.blast') as f:
    for result in BSeIO.parse(f, 'blast-xml'):
        print("Sequences in original: %s" % len(result))
        # print the number of seq and exit the loop
        
# make a new file containing only the filtered results
result_filtered = result.hsp_filter(fn)
# print how many seq's are left
print("Sequences in filtered: %s" % len(result_filtered))
# haltime info to see that things have changed


Sequences in original: 26
Sequences in filtered: 10


In [6]:
# take the ID's from the accession field 
# and append them to a new list
accessions = []
for hit in result_filtered:
    accessions.append(hit.accession)

# take the accession numbers from list and join them with space
# Uniprot only allows space separated IDs
query = " ".join(accessions)
print(query)
# just for halftime info about what we are searching

P0ACS5 Q9HV30 Q93CH6 Q8Z8S3 Q8ZCA8 Q8FK74 Q8XD09 P0A9G4 P0A2Q8 Q9X5V4


In [ ]:
import requests as R
# form an Uniprot query

# API address
url = 'https://www.uniprot.org/uploadlists/'

# required parameters as dictionary
data = {
    # map within Uniprot, to AC/ID
    'from': 'ACC+ID',
    # to Uniprot AC
    'to': 'ACC',
    # output format is xml as GO info from dbxrefs is needed
    'format': 'xml',
    # variable query contains the ID's to search
    'query': query,
   
}

# send query and get response
response = R.get(url, params=data)

#save the response to a mysteryGO.xml -file
with open('mysteryGO.xml', 'w') as f:
    f.write(response.text)

In [16]:
# The following code is from example solution to ex 4.2 
# in the course BIOI4270 provided by teacher Juho Heimonen

# It is used for simplicity and for correctness, as the solution 
# that the author made to the 4.2 was longer and possibly erroneous

# again had to re-import the modules
import Bio.SeqIO as BSqIO
import GOtools as GOT 
go = GOT.parse_go_obo('go.obo')

# init an empty list to gather the terms
molfun = []

# open the previously downloaded file and make a BSIO object of it

for rec in BSqIO.parse("mysteryGO.xml", "uniprot-xml"):

# start iteration from the dbxref section that contains
# the additional info, including GO terms
    for ref in rec.dbxrefs:
    # see if the iteratable item starts with GO:
        if ref.startswith("GO:"):
        # variable takes letters starting from
        # python index 3, as they are format GO:GO...
            item = ref[3:]
        # finally check if the namespace of the GO entry
        # is molecular_function (area of interest)      
            if(go[item].namespace == 'molecular_function'):
                # and if True, append to the list
                molfun.append(item)





In [31]:
# import counter to get the number of occurences
from collections import Counter

# make a new list by implementing the Counter.most_common
# conditions are left empty --> sorts and returns all terms with occurences 
sortMolfun = Counter(molfun).most_common()

# create numbers between 0 and length of the sorted list
for x in range(0,len(sortMolfun)):
    # use that number to retrieve the info from GO and get the occurences of given position
    print("ID:", sortMolfun[x][0], "| Name:", go[sortMolfun[x][0]].name, "| Occurences:", str(sortMolfun[x][1]))
    # ...[x][0] for GO's and ...[x][1] for numbers
    # use the sorted info to get things in right order
    
    # finally figured out that you can use comma in Pyton print()
    # better now than never

ID: GO:0003700 | Name: DNA-binding transcription factor activity | Occurences: 9
ID: GO:0005507 | Name: copper ion binding | Occurences: 8
ID: GO:0003677 | Name: DNA binding | Occurences: 7
ID: GO:0000986 | Name: bacterial-type cis-regulatory region sequence-specific DNA binding | Occurences: 2
ID: GO:0001216 | Name: DNA-binding transcription activator activity | Occurences: 2
ID: GO:0000976 | Name: transcription regulatory region sequence-specific DNA binding | Occurences: 2
ID: GO:0008270 | Name: zinc ion binding | Occurences: 1
ID: GO:0042802 | Name: identical protein binding | Occurences: 1
ID: GO:0045340 | Name: mercury ion binding | Occurences: 1
